<a href="https://colab.research.google.com/github/maniaces/open-vaccine/blob/main/fcn_gru_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow.keras.layers as L
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Set seed to ensure reproducibility

In [ ]:
tf.random.set_seed(2020)
np.random.seed(2020)

## Helper functions and useful variables

In [ ]:
# This will tell us the columns we are predicting
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']

In [ ]:
y_true = tf.random.normal((32, 68, 3))
y_pred = tf.random.normal((32, 68, 3))

In [ ]:
def MCRMSE(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

In [ ]:
def gru_layer(hidden_dim, dropout):
    return L.Bidirectional(L.GRU(
        hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))

def lstm_layer(hidden_dim, dropout):
    return L.Bidirectional(L.LSTM(
        hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))


In [ ]:
def pandas_list_to_array(df):
    """
    Input: dataframe of shape (x, y), containing list of length l
    Return: np.array of shape (x, l, y)
    """
    
    return np.transpose(
        np.array(df.values.tolist()),
        (0, 2, 1)
    )

In [ ]:
def preprocess_inputs(df, token2int, cols=['sequence', 'structure', 'predicted_loop_type']):
    return pandas_list_to_array(
        df[cols].applymap(lambda seq: [token2int[x] for x in seq])
    )

## Load and preprocess data

In [ ]:
data_dir = '/kaggle/input/stanford-covid-vaccine/'
train = pd.read_json(data_dir + 'train.json', lines=True)
test = pd.read_json(data_dir + 'test.json', lines=True)
sample_df = pd.read_csv(data_dir + 'sample_submission.csv')

In [ ]:
train = train.query("signal_to_noise >= 1")

In [ ]:
# We will use this dictionary to map each character to an integer
# so that it can be used as an input in keras
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}

train_inputs = preprocess_inputs(train, token2int)
train_labels = pandas_list_to_array(train[pred_cols])

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    train_inputs, train_labels, test_size=.1, random_state=34, stratify=train.SN_filter)

Public and private sets have different sequence lengths, so we will preprocess them separately and load models of different tensor shapes.

In [ ]:
public_df = test.query("seq_length == 107")
private_df = test.query("seq_length == 130")

public_inputs = preprocess_inputs(public_df, token2int)
private_inputs = preprocess_inputs(private_df, token2int)

## Build and train model



In [ ]:
from keras.layers import Conv1D, BatchNormalization,GlobalMaxPooling1D, Permute, Dropout
from keras.layers import Input, Dense, LSTM, concatenate, Activation
from keras.models import Model

def build_model(embed_size, seq_len=107, pred_len=68, dropout=0.5, 
                sp_dropout=0.2, embed_dim=200, hidden_dim=256, n_layers=16):
    inputs = L.Input(shape=(seq_len, 3))
    embed = L.Embedding(input_dim=embed_size, output_dim=embed_dim)(inputs)
    
    reshaped = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3])
    )
    hiddenn = L.SpatialDropout1D(sp_dropout)(reshaped)
    m_layers= 5
    for y in range(m_layers):
        h = lstm_layer(hidden_dim, dropout)(hiddenn)
        h = Conv1D(512, 5, padding='same', activation=tf.keras.activations.swish)(h)
        h = BatchNormalization()(h)
        hiddenn = Activation('relu')(h)
        
    y = lstm_layer(hidden_dim*2, dropout)(h) 
    x = concatenate([h, y])
    
    y = gru_layer(hidden_dim, dropout)(x)
    z = Conv1D(512, 5, padding='same', activation=tf.keras.activations.swish)(y)
    h = BatchNormalization()(h)
    hiddenn = Activation('relu')(h)
    z = Conv1D(512, 5, padding='same', activation=tf.keras.activations.swish)(h)    
    z = Conv1D(512, 5, padding='same', activation=tf.keras.activations.swish)(z)
    z = gru_layer(hidden_dim, dropout)(z)
    z = lstm_layer(hidden_dim*2, dropout)(z)
    xx = concatenate([x, z])
    y = gru_layer(hidden_dim, dropout)(xx)
    z = Conv1D(512, 5, padding='same', activation=tf.keras.activations.swish)(y)
    z = gru_layer(hidden_dim*4, dropout)(z)
    z = lstm_layer(hidden_dim*2, dropout)(z)
    
    
    # Since we are only making predictions on the first part of each sequence, 
    # we have to truncate it
    truncated = z[:, :pred_len]
    out = L.Dense(5, activation='linear')(truncated)
    
    model = tf.keras.Model(inputs=inputs, outputs=out)
    model.compile(tf.optimizers.Adam(), loss=MCRMSE)
    
    return model

In [ ]:
model = build_model(embed_size=len(token2int))
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 107, 3)]     0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 107, 3, 200)  2800        input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Reshape (TensorFlow [(None, 107, 600)]   0           embedding[0][0]                  
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 107, 600)     0           tf_op_layer_Reshape[0][0]        
_______________________________________________________________________________________

In [ ]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    batch_size=64,
    epochs=200,
    verbose=2,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(patience=5),
        tf.keras.callbacks.ModelCheckpoint('model.h5')
    ]
)

Epoch 1/200
30/30 - 25s - loss: 0.7946 - val_loss: 0.4746
Epoch 2/200
30/30 - 20s - loss: 0.4486 - val_loss: 0.4538
Epoch 3/200
30/30 - 19s - loss: 0.4226 - val_loss: 0.4493
Epoch 4/200
30/30 - 19s - loss: 0.4074 - val_loss: 0.4198
Epoch 5/200
30/30 - 19s - loss: 0.3830 - val_loss: 0.4287
Epoch 6/200
30/30 - 19s - loss: 0.3682 - val_loss: 0.4140
Epoch 7/200
30/30 - 19s - loss: 0.3592 - val_loss: 0.4060
Epoch 8/200
30/30 - 19s - loss: 0.3521 - val_loss: 0.3950
Epoch 9/200
30/30 - 19s - loss: 0.3481 - val_loss: 0.3964
Epoch 10/200
30/30 - 19s - loss: 0.3402 - val_loss: 0.3864
Epoch 11/200
30/30 - 19s - loss: 0.3402 - val_loss: 0.3739
Epoch 12/200
30/30 - 19s - loss: 0.3352 - val_loss: 0.3855
Epoch 13/200
30/30 - 19s - loss: 0.3311 - val_loss: 0.3654
Epoch 14/200
30/30 - 19s - loss: 0.3257 - val_loss: 0.3591
Epoch 15/200
30/30 - 19s - loss: 0.3201 - val_loss: 0.3456
Epoch 16/200
30/30 - 19s - loss: 0.3146 - val_loss: 0.3273
Epoch 17/200
30/30 - 19s - loss: 0.3099 - val_loss: 0.3137
Epoch 

## Evaluate training history

Let's use Plotly to quickly visualize the training and validation loss throughout the epochs.

In [ ]:
fig = px.line(
    history.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'MCRMSE'}, 
    title='Training History')
fig.show()

## Load models and make predictions

Public and private sets have different sequence lengths, so we will preprocess them separately and load models of different tensor shapes. This is possible because RNN models can accept sequences of varying lengths as inputs.

In [ ]:
# Caveat: The prediction format requires the output to be the same length as the input,
# although it's not the case for the training data.
model_public = build_model(seq_len=107, pred_len=107, embed_size=len(token2int))
model_private = build_model(seq_len=130, pred_len=130, embed_size=len(token2int))

model_public.load_weights('model.h5')
model_private.load_weights('model.h5')

In [ ]:
public_preds = model_public.predict(public_inputs)
private_preds = model_private.predict(private_inputs)

## Post-processing and submit

For each sample, we take the predicted tensors of shape (107, 5) or (130, 5), and convert them to the long format (i.e. $629 \times 107, 5$ or $3005 \times 130, 5$):

In [ ]:
preds_ls = []

for df, preds in [(public_df, public_preds), (private_df, private_preds)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)
preds_df.head()

,reactivity,deg_Mg_pH10,deg_Mg_50C,deg_pH10,deg_50C,id_seqpos
0,0.566195,0.615158,0.504208,1.865444,0.761384,id_00073f8be_0
1,1.930257,2.796616,2.967012,4.224547,2.804580,id_00073f8be_1
2,1.391189,0.550995,0.533514,0.681997,0.620566,id_00073f8be_2
3,1.201813,1.119557,1.619588,1.244817,1.668552,id_00073f8be_3
4,0.727673,0.509008,0.752700,0.466805,0.766629,id_00073f8be_4


In [ ]:
submission = sample_df[['id_seqpos']].merge(preds_df, on=['id_seqpos'])
submission.to_csv('submission.csv', index=False)